In [1]:
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import torch.nn as nn
from matplotlib import pyplot as plt
torch.cuda.is_available()
#Daniel Sullivan
#801095863

C:\Users\rando\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\rando\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


True

In [2]:
import pandas as pd
import torch.optim as optim


housing = pd.DataFrame(pd.read_csv("Housing.csv")) 
housing.head() 

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking','price'] 
Newtrain = housing[num_vars] 
Newtrain.head() 

,area,bedrooms,bathrooms,stories,parking,price
0,7420,4,2,3,2,13300000
1,8960,4,4,4,3,12250000
2,9960,3,2,2,2,12250000
3,7500,4,2,2,3,12215000
4,7420,4,1,2,2,11410000


In [4]:
from sklearn.model_selection import train_test_split 
 
# We specify this so that the train and test data set always have the same rows, respec
np.random.seed(0) 
df_train, df_test = train_test_split(Newtrain, train_size = 0.8, test_size = 0.2, random_state=42)

In [5]:
trainArray = df_train.to_numpy()
testArray = df_test.to_numpy()


inputTrain = trainArray[:, 0:5]
outputTrain = trainArray[:, 5]

inputTest = testArray[:, 0:5]
outputTest = testArray[:, 5]

inputTrainT = torch.tensor(inputTrain)
outputTrainT = torch.tensor(outputTrain)

inputTestT = torch.tensor(inputTest)
outputTestT = torch.tensor(outputTest)

inputTrainT = inputTrainT.to(torch.float32)
outputTrainT = outputTrainT.to(torch.float32)

inputTestT = inputTestT.to(torch.float32)
outputTestT = outputTestT.to(torch.float32)

In [6]:
inTrainMean = torch.mean(inputTrainT)
outTrainMean = torch.mean(outputTrainT)

inTrainStd = torch.std(inputTrainT)
outTrainStd = torch.std(outputTrainT)

inputTrainT = (inputTrainT - inTrainMean) / inTrainStd
outputTrainT = (outputTrainT - outTrainMean) / outTrainStd


outputTrainT = torch.tensor(outputTrainT).unsqueeze(1)
outputTestT = torch.tensor(outputTestT).unsqueeze(1)
outputTrainT.shape

C:\Users\rando\AppData\Local\Temp/ipykernel_10112/2610806285.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputTrainT = torch.tensor(outputTrainT).unsqueeze(1)
C:\Users\rando\AppData\Local\Temp/ipykernel_10112/2610806285.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputTestT = torch.tensor(outputTestT).unsqueeze(1)


torch.Size([436, 1])

In [7]:
#Problem 1 Part A

seq_model = nn.Sequential(
            nn.Linear(5, 8),
            nn.ReLU(),
            nn.Linear(8,1))

seq_model_2 = nn.Sequential(
            nn.Linear(5, 8),
            nn.ReLU(),
            nn.Linear(8, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 1))

learningRate = 0.01
optimizer = optim.SGD(seq_model.parameters(), lr = learningRate)
seq_model

Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [8]:
def trainingLoop(numEpochs, optimizer, model, loss_fn, trainInput, trainOutput, valInput, valOutput):
    for epoch in range(1, numEpochs + 1):
        t_p_train = model(trainInput)
        loss_train = loss_fn(t_p_train, trainOutput)
        t_p_val = model(valInput)
    
        loss_val = loss_fn(t_p_val, valOutput)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if (epoch > 100) and (epoch < 200) or (epoch % 1000) == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")
            

In [9]:
trainingLoop(2000, optimizer, seq_model, nn.MSELoss(), inputTrainT, outputTrainT, inputTestT, outputTestT)

In [28]:
import torchvision.models as models
from ptflops import get_model_complexity_info

with torch.cuda.device(0):
  macs, params = get_model_complexity_info(seq_model, (1, 5), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))


Sequential(
  0.0 M, 100.000% Params, 0.0 GMac, 100.000% MACs, 
  (0): Linear(0.0 M, 84.211% Params, 0.0 GMac, 73.846% MACs, in_features=5, out_features=8, bias=True)
  (1): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 12.308% MACs, )
  (2): Linear(0.0 M, 15.789% Params, 0.0 GMac, 13.846% MACs, in_features=8, out_features=1, bias=True)
)
Computational complexity:       0.0 GMac
Number of parameters:           57      


In [11]:
learningRate = 0.05
optimizer = optim.SGD(seq_model_2.parameters(), lr = learningRate)

trainingLoop(3000, optimizer, seq_model_2, nn.MSELoss(), inputTrainT, outputTrainT, inputTestT, outputTestT)

In [27]:
with torch.cuda.device(0):
  macs, params = get_model_complexity_info(seq_model_2, (1, 5), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))


Sequential(
  0.0 M, 100.000% Params, 0.0 GMac, 100.000% MACs, 
  (0): Linear(0.0 M, 37.209% Params, 0.0 GMac, 32.653% MACs, in_features=5, out_features=8, bias=True)
  (1): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 5.442% MACs, )
  (2): Linear(0.0 M, 34.884% Params, 0.0 GMac, 30.612% MACs, in_features=8, out_features=5, bias=True)
  (3): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 3.401% MACs, )
  (4): Linear(0.0 M, 23.256% Params, 0.0 GMac, 20.408% MACs, in_features=5, out_features=5, bias=True)
  (5): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 3.401% MACs, )
  (6): Linear(0.0 M, 4.651% Params, 0.0 GMac, 4.082% MACs, in_features=5, out_features=1, bias=True)
)
Computational complexity:       0.0 GMac
Number of parameters:           129     


In [13]:
#Problem 2

In [14]:
from torchvision import datasets
data_path = 'C:/Users/rando/Desktop/Real Time ML/HW2'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform = transforms.ToTensor())
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4195, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))

Files already downloaded and verified
Files already downloaded and verified


In [15]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)


cifarModel = nn.Sequential(
            nn.Linear(3072, 512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.LogSoftmax(dim=1))
cifarModel.cuda()


Sequential(
  (0): Linear(in_features=3072, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=10, bias=True)
  (3): LogSoftmax(dim=1)
)

In [16]:
learning_rate = 0.01
optimizer = optim.SGD(cifarModel.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 350

In [17]:
for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        imgs = imgs.cuda()
        labels = labels.cuda()
        
        batch_size = imgs.shape[0]
        outputs = cifarModel(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch % 25) == 0:    
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 2.045917
Epoch: 25, Loss: 1.805457
Epoch: 50, Loss: 0.884993
Epoch: 75, Loss: 1.566544
Epoch: 100, Loss: 1.367481
Epoch: 125, Loss: 0.615263
Epoch: 150, Loss: 0.380259
Epoch: 175, Loss: 0.437532
Epoch: 200, Loss: 0.638911
Epoch: 225, Loss: 0.212790
Epoch: 250, Loss: 0.073755
Epoch: 275, Loss: 0.252923
Epoch: 300, Loss: 0.061228
Epoch: 325, Loss: 0.176649


In [18]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.cuda()
        labels = labels.cuda()
        batch_size = imgs.shape[0]
        outputs = cifarModel(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted==labels).sum())
        
print("Accuracy: %f", correct/total)

Accuracy: %f 0.3553


In [26]:
with torch.cuda.device(0):
  macs, params = get_model_complexity_info(cifarModel, (1, 3072), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))


Sequential(
  1.579 M, 100.000% Params, 0.002 GMac, 100.000% MACs, 
  (0): Linear(1.573 M, 99.675% Params, 0.002 GMac, 99.643% MACs, in_features=3072, out_features=512, bias=True)
  (1): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 0.032% MACs, )
  (2): Linear(0.005 M, 0.325% Params, 0.0 GMac, 0.325% MACs, in_features=512, out_features=10, bias=True)
  (3): LogSoftmax(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, dim=1)
)
Computational complexity:       0.0 GMac
Number of parameters:           1.58 M  


In [20]:
cifarModel_2 = nn.Sequential(
            nn.Linear(3072, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
            nn.LogSoftmax(dim=1))
cifarModel_2.cuda()

Sequential(
  (0): Linear(in_features=3072, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=512, bias=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=10, bias=True)
  (7): LogSoftmax(dim=1)
)

In [21]:
learning_rate = 0.01
optimizer = optim.SGD(cifarModel_2.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 350

In [22]:
for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        imgs = imgs.cuda()
        labels = labels.cuda()
        
        batch_size = imgs.shape[0]
        outputs = cifarModel_2(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch % 25) == 0:    
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 2.052646
Epoch: 25, Loss: 1.133580
Epoch: 50, Loss: 0.637191
Epoch: 75, Loss: 0.460126
Epoch: 100, Loss: 0.125307
Epoch: 125, Loss: 0.828143
Epoch: 150, Loss: 0.006734
Epoch: 175, Loss: 0.000433
Epoch: 200, Loss: 0.001567
Epoch: 225, Loss: 0.001868
Epoch: 250, Loss: 0.000816
Epoch: 275, Loss: 0.001290
Epoch: 300, Loss: 0.001010
Epoch: 325, Loss: 0.001172


In [23]:
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.cuda()
        labels = labels.cuda()
        batch_size = imgs.shape[0]
        outputs = cifarModel_2(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted==labels).sum())
        
print("Accuracy: %f", correct/total)

Accuracy: %f 0.3807


In [25]:
with torch.cuda.device(0):
  macs, params = get_model_complexity_info(cifarModel_2, (1, 3072), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Sequential(
  1.903 M, 100.000% Params, 0.002 GMac, 100.000% MACs, 
  (0): Linear(1.573 M, 82.679% Params, 0.002 GMac, 82.629% MACs, in_features=3072, out_features=512, bias=True)
  (1): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 0.027% MACs, )
  (2): Linear(0.263 M, 13.802% Params, 0.0 GMac, 13.794% MACs, in_features=512, out_features=512, bias=True)
  (3): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 0.027% MACs, )
  (4): Linear(0.066 M, 3.451% Params, 0.0 GMac, 3.448% MACs, in_features=512, out_features=128, bias=True)
  (5): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 0.007% MACs, )
  (6): Linear(0.001 M, 0.068% Params, 0.0 GMac, 0.068% MACs, in_features=128, out_features=10, bias=True)
  (7): LogSoftmax(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, dim=1)
)
Computational complexity:       0.0 GMac
Number of parameters:           1.9 M   
